In [19]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm, trange

import time

In [2]:
# Using scrip file
# Find ncol given lat and lon
from SE_analysis import get_site_index # To get index


# Read in scrip file
ds_rr_scrip = "./Data/conus_30_x8.g_scrip.nc"

# Specify location, e.g. find column index at SOR - lat 35N and lon 253.5 E
SOR_lonlat = [253.5, 35]

# call get_site_index 
Index = get_site_index(SOR_lonlat[0],SOR_lonlat[1],scrip_file=ds_rr_scrip)
print(Index)

116909


---

In [4]:
# using own script

# example script
ds = xr.open_dataset("./Data/ignore_data/FWmaHIST_ne30pg3_ne30pg3_mg17_no_conus_gravity_waves.cam.h0.2010-01-01-00000.nc")
ds.ncol

<xarray.DataArray 'ncol' (ncol: 48602)>
array([    0,     1,     2, ..., 48599, 48600, 48601])
Dimensions without coordinates: ncol

In [27]:
candidate = [] 
for col in trange(len(ds.ncol)):
    perc_done = 100*col/len(ds.ncol)
    # print("% complete " , perc_done, end="\r")
    dlat = np.abs(35 - ds.isel(ncol = col)['lat'].values)
    dlon = np.abs(253 - ds.isel(ncol = col)['lon'].values)
    if (dlat < 2) & (dlon < 2):
        print("candidate found: ", col, end="\n")
        candidate.append(col)



  0%|          | 0/48602 [00:00<?, ?it/s]

candidate found:  31768
candidate found:  31769
candidate found:  31771
candidate found:  31772
candidate found:  31776
candidate found:  31777
candidate found:  31779
candidate found:  31780
candidate found:  32032
candidate found:  32033
candidate found:  32035
candidate found:  32036
candidate found:  32040
candidate found:  32041
candidate found:  32043


In [42]:
# print col, lat, lon and inspect for nearest
# I'm sure there's a more python way of doing this
for col in candidate:
    lat = ds.isel(ncol=col)['lat'].values
    lon = ds.isel(ncol=col)['lon'].values
    print("col ", col, " lat ", lat, " lon ", lon)

---

In [48]:
# slice level to reduce data
# Note - this does not give me the actual column index, it just gives me the index 
# from the list of reduced indices

# lat = lat_val
# lon = lon_val

# extract dataset over small region surrounding SOR using "where"
data_isel_lev = ds.isel(lev=0)

data_reduced = data_isel_lev.where((data_isel_lev['lat'] < 36)  & \
                                   (data_isel_lev['lat'] > 34)  & \
                                   (data_isel_lev['lon'] < 255) & \
                                   (data_isel_lev['lon'] > 250), 
                                   drop=True)

#inspect
data_reduced
# loop to find candidates using e.g. 
# lat - lat_val < "0.01" 
# lon - lon_val < "0.01"

<xarray.Dataset>
Dimensions:       (ncol: 9, ilev: 111, time: 24, nbnd: 2)
Coordinates:
    lev           float64 6.14e-06
  * ilev          (ilev) float64 4.949e-06 7.618e-06 1.173e-05 ... 985.1 1e+03
  * time          (time) object 2010-01-01 00:00:00 ... 2010-01-01 23:00:00
Dimensions without coordinates: ncol, nbnd
Data variables: (12/44)
    lat           (ncol) float64 34.51 34.64 34.13 34.77 ... 35.46 35.58 35.77
    lon           (ncol) float64 251.2 252.0 254.2 252.8 ... 252.0 252.8 254.2
    area          (ncol) float64 0.000154 6.162e-05 ... 0.0003818 0.0003817
    hyam          (ncol) float64 6.14e-09 6.14e-09 ... 6.14e-09 6.14e-09
    hybm          (ncol) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
    hyai          (ilev, ncol) float64 4.949e-09 4.949e-09 4.949e-09 ... 0.0 0.0
    ...            ...
    OMEGA         (time, ncol) float32 3.062e-08 1.803e-08 ... 4.241e-08
    PS            (time, ncol) float32 7.786e+04 7.766e+04 ... 7.814e+04
    T             (time, ncol) float32 559.3 563.0 566.6 ... 571.0 563.4 540.3
    TROP_P        (time, ncol) float32 3.024e+04 2.995e+04 ... 2.885e+04
    U             (time, ncol) float32 -8.704 -25.83 -28.72 ... -37.36 -36.56
    V             (time, ncol) float32 -65.94 -70.64 -79.94 ... 119.4 143.0
Attributes:
    ne:                30
    np:                4
    Conventions:       CF-1.0
    source:            CAM
    case:              FWmaHIST_ne30pg3_ne30pg3_mg17_no_conus_gravity_waves
    logname:           nadavis
    host:              cheyenne2
    initial_file:      /glade/scratch/nadavis/cesm2_inputs/f.e21.FWHIST.110L....
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/se/ne30n...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  hour_1